### Pobieranie modelu 
Link do manualnego pobrania: https://www.comet.ml/plutasnyy/model-registry/emd2-bert?search=1.0.0

In [1]:
!pip install comet-ml pytorch-lightning transformers

     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 686kB 13.8MB/s 
     |████████████████████████████████| 1.5MB 23.1MB/s 
     |████████████████████████████████| 204kB 40.8MB/s 
     |████████████████████████████████| 512kB 48.3MB/s 
     |████████████████████████████████| 276kB 51.0MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 829kB 50.9MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
     |████████████████████████████████| 2.9MB 51.4MB/s 
     |████████████████████████████████| 1.3MB 44.4MB/s 
     |████████████████████████████████| 143kB 52.0MB/s 
     |████████████████████████████████| 296kB 48.6MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=77eb6038c74a767024919e0b7ab89e676ae3de9e6cbab8293e80c0c22117e0a2
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Cre

In [2]:
!wget https://www.comet.ml/api/registry/model/item/download?modelItemId=bJJfGgbQ4s266JvhOlv5KpGdF -O model.zip

--2021-01-10 19:05:09--  https://www.comet.ml/api/registry/model/item/download?modelItemId=bJJfGgbQ4s266JvhOlv5KpGdF
Resolving www.comet.ml (www.comet.ml)... 52.204.124.35, 34.196.199.220
Connecting to www.comet.ml (www.comet.ml)|52.204.124.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘model.zip’

model.zip               [      <=>           ] 386.82M  45.9MB/s    in 9.7s    

2021-01-10 19:05:48 (39.7 MB/s) - ‘model.zip’ saved [405611090]



In [3]:
!unzip model.zip

Archive:  model.zip
  inflating: epoch=00-val_loss=0.7696-f1_cum=0.5785.ckpt  


### Wczytanie modelu oraz wag

In [4]:
import pytorch_lightning as pl
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import numpy as np

class LitModule(pl.LightningModule):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = BertForSequenceClassification(
            BertConfig(num_labels=5, output_attentions=False, output_hidden_states=False)
        )

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

model = LitModule.load_from_checkpoint('epoch=00-val_loss=0.7696-f1_cum=0.5785.ckpt')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

comet_ml is installed but `COMET_API_KEY` is not set.


In [6]:
model.eval()
model.freeze()
model.cuda()

### Predict
Plik test.csv to podzbiór pliku z danymi dostarczonymi z zadaniem, u nas on został wykonany za pomocą skryptu: preprocess.ipynb

In [8]:
import pandas as pd
df = pd.read_csv("test.csv")
df = df[['reviewText', 'score']]
df['score'] = df['score'].astype(int)
df.head()

,reviewText,score
0,24 7 has several different themes of Mahjong a...,5
1,I like reading the secrets that people have bu...,3
2,Where's my perry is a fun. it's lots like wher...,5
3,"who loves to shine poo? I do! come on, will yo...",5
4,I give it five stars because it's fun and cool...,5


In [15]:
import torch
from tqdm import tqdm_notebook

y_true, y_pred = list(), list()
for i, row in tqdm_notebook(df.iterrows(), total=len(df)):
    encode = tokenizer(row['reviewText'], add_special_tokens=True, padding='max_length', max_length=128, truncation=True)
    item = {key: torch.tensor(val).long() for key, val in encode.items()}
    
    outputs = model(
        item['input_ids'].unsqueeze(0).cuda(),
        token_type_ids=None,
        attention_mask=item['attention_mask'].unsqueeze(0).cuda()
    )  
    predeicted = np.argmax(outputs.logits.cpu().numpy().squeeze(), axis=-1).astype(int)

    y_true.append(row['score']-1) # model przewiduje od 0-4
    y_pred.append(predeicted)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.680     0.761     0.718      6014
           1      0.362     0.427     0.392      3374
           2      0.476     0.496     0.486      6389
           3      0.475     0.506     0.490     11653
           4      0.843     0.773     0.806     28145

    accuracy                          0.663     55575
   macro avg      0.567     0.593     0.578     55575
weighted avg      0.677     0.663     0.668     55575

